In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

#sales_15 = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/2015_sales_data.csv')
#sales_16 = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/2016_sales_data.csv')
sales_17 = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/2017_sales_data.csv')
sales_18 = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/2018_sales_data.csv')
city     = pd.read_json('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/city_dict.json',typ='Series')
e_dis = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/expected_discount.csv')
h_dis = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/historical_discount.csv')
ff = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/foot_fall.csv')
product = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/product_information.csv')
test = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/test_data.csv')
test_id=pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/test_data.csv')

In [ ]:
city=pd.DataFrame(city)
city.reset_index(level=0, inplace=True)
city.columns = ['city', 'city_name']

In [ ]:
#########################            Train data


Sales = pd.concat([sales_17,sales_18])
Sales_city=Sales.merge(city,how='left',on=['city','city'])
h_dis_pd=h_dis.melt(id_vars=["date", "product"], 
        var_name="city_name", 
        value_name="dis")
h_dis_pd['city_name']=h_dis_pd['city_name'].str.replace('Discount_','')
Sales_dis=pd.merge(Sales_city,h_dis_pd,how='left',left_on=['date','product','city_name'], right_on = ['date','product','city_name'])
Sales_prod=pd.merge(Sales_dis,product,how='left',on=['product','product'])
train=Sales_prod


In [ ]:
train

In [ ]:
train['datetime'] = pd.to_datetime(train['date'])

train['weekday'] = train['datetime'].dt.dayofweek
train['monthday']= train['datetime'].dt.day
train['month']=train['datetime'].dt.month
train['year']=train['datetime'].dt.year
train['weekend']= train['weekday'].apply(lambda x:1 if x>4 else 0 )
#train['sunday_or_monday']= train['weekday'].apply(lambda x:1 if x>5 or x<1 else 0 )
#train['hour']= train['datetime'].apply(lambda x:x.hour )
#train['peak_hour']=train['hour'].apply(lambda x:1 if x<6 or x>19 else 0)
#train['time_filter']=train['hour'].apply(lambda x:1 if x is 2 or x is 4 else 0 )
#train['date_filter']=train['monthday'].apply(lambda x:1 if x is 23 or x is 30 else 0 )
train.drop(['date','city','dis'],axis=1,inplace=True)
train


In [ ]:
ff_train=ff.melt(id_vars=["city"], 
        var_name="datetime", 
        value_name="ff")
ff_train['datetime'] = pd.to_datetime(ff_train['datetime'])


train=train.merge(ff_train,how='left',left_on=['city_name','datetime'],right_on=['city','datetime'])
train

In [ ]:
train.isna().sum()

In [ ]:
train.drop(['datetime','city'],axis=1,inplace=True)
train['product'] = train['product'].astype(object)
train.drop(['var_4','var_7'],axis=1,inplace=True)
train

In [ ]:
train_cat=train.drop(['ff'],axis=1)
train_num=train.drop(['product', 'sales', 'city_name', 'product_category',
       'product_subcategory', 'var_1', 'var_2', 'var_3', 'var_5', 'var_6',
       'var_8', 'var_9', 'var_10', 'weekday', 'monthday', 'month', 'year',
       'weekend'],axis=1)

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(train_num.values)
train_num = imp.transform(train_num.values)
train_num=pd.DataFrame(train_num)
train_num.columns =['ff']
train=train_cat.merge(train_num,left_index=True,right_index=True)

In [ ]:
train_ff_pred=train.drop(['product', 'sales', 'product_category',
       'product_subcategory', 'var_1', 'var_2', 'var_3', 'var_5', 'var_6',
       'var_8', 'var_9', 'var_10'],axis=1)

In [ ]:
train_ff_pred.drop_duplicates(keep='first',inplace=True)
train_ff_pred

In [ ]:
train.to_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Outputs/train_v4(lgb for ff pred_nodis).csv',index=False)

In [ ]:
###########          Test


test_city=test.merge(city,how='left',on=['city','city'])
e_dis_pd=e_dis.melt(id_vars=["date", "product"], 
        var_name="city_name", 
        value_name="dis")

e_dis_pd['city_name']=e_dis_pd['city_name'].str.replace('Discount_','')
test_dis=pd.merge(test_city,e_dis_pd,how='left',left_on=['date','product','city_name'], right_on = ['date','product','city_name'])
test_prod=pd.merge(test_dis,product,how='left',on=['product','product'])
test=test_prod


In [ ]:
test['datetime'] = pd.to_datetime(test['date'])

test['weekday'] = test['datetime'].dt.dayofweek
test['monthday']= test['datetime'].dt.day
test['month']=test['datetime'].dt.month
test['year']=test['datetime'].dt.year
test['weekend']= test['weekday'].apply(lambda x:1 if x>4 else 0 )
#test['sunday_or_monday']= test['weekday'].apply(lambda x:1 if x>5 or x<1 else 0 )
#test['hour']= test['datetime'].apply(lambda x:x.hour )
#test['peak_hour']=test['hour'].apply(lambda x:1 if x<6 or x>19 else 0)
#test['time_filter']=test['hour'].apply(lambda x:1 if x is 2 or x is 4 else 0 )
#test['date_filter']=test['monthday'].apply(lambda x:1 if x is 23 or x is 30 else 0 )
#test.to_csv('C:/Users/np8022/Desktop/analytics Vidhya competition/JH Demand Forecasting/Outputs/test_download.csv',index=False)
#test




In [ ]:
############ LGB for FF pred, don't use for Arima  ################### 



test_ff_lgb=test.drop(['id', 'date', 'product', 'dis', 'product_category',
       'product_subcategory', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5',
       'var_6', 'var_7', 'var_8', 'var_9', 'var_10','datetime','city'],axis=1)
test_ff_lgb.drop_duplicates(keep='first',inplace=True)


from lightgbm import LGBMRegressor
Xd,yd=train_ff_pred.drop('ff',axis=1),train_ff_pred.ff

for col in ['city_name']:
            Xd[col] = Xd[col].astype('category')
            test_ff_lgb[col] = test_ff_lgb[col].astype('category')


lgbm_ff = LGBMRegressor()
lgbm_ff.fit(Xd,yd)

test_ff_lgb['ff']= lgbm_ff.predict(test_ff_lgb)

test=test.merge(test_ff_lgb,how='left',on=['city_name','monthday','month','year','weekday','weekend'])
test.drop(['id', 'date', 'city','dis','datetime'],axis=1,inplace=True)

In [ ]:
############ ARIMA for FF pred, don't use for LGB  ###################



test_ff=test.drop(['id', 'date', 'city', 'product', 'dis', 'product_category',
       'product_subcategory', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5',
       'var_6', 'var_7', 'var_8', 'var_9', 'var_10', 'weekday',
       'monthday', 'month', 'year', 'weekend'],axis=1)

test_ff_unq=test_ff.groupby(['city_name','datetime']).size().reset_index().rename(columns={0:'count'})
test_ff_unq.drop('count',axis=1,inplace=True)
test_ff_unq=test_ff_unq.sort_values(by=['datetime','city_name'])

ff_pd=ff.melt(id_vars=["city"], 
        var_name="date", 
        value_name="ff")
ff_pd=ff_pd.sort_values(by=['date','city'])

ff_pd['datetime'] = pd.to_datetime(ff_pd['date'])
ff_pd.drop(['date'],axis=1,inplace=True)
ff_pd.set_index('datetime',inplace=True)
ff_pd=ff_pd.dropna()

city_list=ff_pd['city'].unique().tolist()

In [ ]:
from pmdarima import auto_arima

ff_test_pred= pd.DataFrame(columns=['date','city', 'ff'])

for item in city_list:
    train_model=ff_pd[ff_pd['city']==item]
    test_filtered=test_ff_unq[test_ff_unq['city_name']==item]
    train_model.drop('city',axis=1,inplace=True)

#final_table= pd.DataFrame(columns=['Course_ID', 'Sales', 'Day_No'])
#Day_No=range(883,943)
#for ID in range(1,139):
#    train_time=train_date[train_date['Course_ID']==ID]
#    train_time=train_time[['date','Sales']]
#    train_time['date'] = pd.to_datetime(train_time['date'])
#    train_time.set_index('date',inplace=True)
   
    model = auto_arima(train_model, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           #parallel=True,n_jobs=8,
                           suppress_warnings=True, 
                           stepwise=True)
    print(model.aic())
    print(item)
    model.fit(train_model)
    
    
    forecast = model.predict(n_periods=len(test_filtered))
    
    s=pd.DataFrame({'date':test_filtered.datetime,'city':test_filtered.city_name,'ff':forecast})
    #print(s)
    ff_test_pred=ff_test_pred.append(s,ignore_index=True)

ff_test_pred.to_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Outputs/Sarimax 7 v1.csv',index=False)

    
    #forecast = pd.DataFrame(forecast,index = (883:942),columns=['Prediction'])
#print(train_time.head())
#train_time.dtypes

test=test.merge(ff_test_pred,how='left',left_on=['city_name','datetime'],right_on=['city','date'])
test.drop(['id','datetime','date_x','city_x','dis','date_y','city_y'],axis=1,inplace=True)
test['product'] = test['product'].astype(object)

In [ ]:
############ Regular code for test begins, use for both arima and LGB for ff prediction   ###################

In [ ]:
test['product_category'].fillna(value=test['product_category'].value_counts().index[0],inplace =True)
test['product_subcategory'].fillna(value=test['product_subcategory'].value_counts().index[0],inplace =True)


In [ ]:

test_dropped=test.drop(['product_category','product_subcategory','product','city_name','weekday','monthday','month','year','weekend','ff'],axis=1)
test_cat=test.drop(['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'var_8',
       'var_9', 'var_10'],axis=1)


In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(test_dropped.values)
test_dropped = imp.transform(test_dropped.values)
test_dropped=pd.DataFrame(test_dropped)
test_dropped.columns =['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'var_8',
       'var_9', 'var_10']

In [ ]:
test=test_dropped.merge(test_cat,left_index=True, right_index=True)


In [ ]:
test.drop(['var_4','var_7'],axis=1,inplace=True)

In [ ]:
test

In [ ]:
test.to_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Outputs/test_v4(lgb for ff pred_nodis).csv',index=False)

In [2]:
train = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Outputs/train_v4(lgb for ff pred_nodis).csv')
test = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Outputs/test_v4(lgb for ff pred_nodis).csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
train

In [3]:
train.dtypes

product                  int64
sales                  float64
city_name               object
product_category        object
product_subcategory     object
var_1                  float64
var_2                  float64
var_3                  float64
var_5                  float64
var_6                  float64
var_8                  float64
var_9                  float64
var_10                 float64
weekday                  int64
monthday                 int64
month                    int64
year                     int64
weekend                  int64
ff                     float64
dtype: object

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import fbeta_score, make_scorer
MSE = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
def RMSE(a,b): 
  return np.sqrt(((a - b) ** 2).mean())
from sklearn.metrics import make_scorer
RMSE_score = make_scorer(RMSE, greater_is_better=False)

In [ ]:
param_test ={'learning_rate': [0.01,0.1],
             'subsample': [0.4,0.7], 
             'colsample_bytree': [0.7,0.9],
             'reg_lambda': [0.1,10]}

In [ ]:
param_test_temp ={'learning_rate': [0.01,0.1]}

In [ ]:
#from scipy.stats import randint as sp_randint
#from scipy.stats import uniform as sp_uniform
#param_test ={'learning_rate': [0.01,0.1,0.8,0.08],
             'num_leaves': [6,15,50], 
             'min_child_samples': [100,300,500], 
             'min_child_weight': [1e-4, 1e-1, 1, 1e3],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}


In [ ]:
#############   tuning output  #################
params_tuned= {'colsample_bytree': 0.8666214463442921, 'learning_rate': 0.8, 'min_child_samples': 100, 'min_child_weight': 1, 'num_leaves': 50, 'reg_alpha': 5, 'reg_lambda': 0.1, 'subsample': 0.9780808911239147}

In [ ]:
from lightgbm import LGBMRegressor

Xd,yd=train.drop('sales',axis=1),train.sales
for col in ['product_category','product_subcategory','product','city_name']:
            Xd[col] = Xd[col].astype('category')
            test[col] = test[col].astype('category')

categorical_features_indices = np.where(Xd.dtypes =='object')[0]


lgbm = LGBMRegressor(categorical_features = categorical_features_indices,metric='RMSE',random_state=13)


iter = 2


from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
gs = RandomizedSearchCV(
    estimator=lgbm, param_distributions=param_test_temp, 
    n_iter=iter,
    scoring=RMSE_score,
    cv=2,
    refit=True,
    random_state=13,
    n_jobs = 1,
    verbose=100)
gs.fit(Xd, yd)
print('Best score: {} params: {} '.format(gs.best_score_, gs.best_params_))

pred=[]
pred = gs.predict(test)

In [ ]:

test_id=pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Inputs/test_data.csv')

s=pd.DataFrame({'id':test_id.id,'sales':pred})
s=s.round(2)


s.to_csv('C:/Users/np8022/Desktop/ZS Hack/ADS TT/Outputs/LGBM 5_LGB_ffpred_notuning(-73 val rmse learn0.1).csv',index=False)
s.head()